# PROBLEM STATEMENT

This very strange transmission is coming from your narrowband radio signal receiver, pointed towards one of the farthest away galaxies. It’s early morning, you are sitting in your radio observatory high in the mountains.

For the last 10 years you’ve been a Chief Data Scientist in one of the best astrophysics research teams in the world. You are enjoying a quiet time with a cup of coffee and reviewing the data reports from last night, when this strange sound arrived. You almost spill your coffee in surprise. “Am I dreaming?” is your first thought as you move closer towards the speaker and listen…

“Beep…Beeeep….Beeeep… To all Hooomans who can hear us – we need your help”

You lean closer and grab a notebook and a pencil – you don’t really trust computers when it comes to such important tasks as taking notes from a radio transmission. You start recording everything that the strange voice from light years away is saying.

“… We need serious Data Science help and we know you Hooomans are the best at it…. We are an intergalactic species which have almost achieved singularity and the highest possible levels of development. We travel fast through space and explore other galaxies”

“The only essence that we consume is energy, measured in DSML units…Our populace is widespread and we live across many different star clusters and galaxies. What we need now is to optimize our well-being across all those galaxies… We have a lot of data but our сomputers and methods are too weak – we urgently need your data science knowledge to help us”

“Only two steps prevent us from achieving singularity

· To understand what makes us better off.

Our elders used the composite index to measure our well-being performance, but this knowledge has disappeared in the sands of time.

Use our data and train your model to predict this index with the highest possible level of certainty.

· To achieve the highest possible level of well-being through optimized allocation of additional energy

We have discovered the star of an unusually high energy of 50000 zillion DSML.

We have agreed between ourselves that 

· no one galaxy will consume more than 100 zillion DSML 

and 

· at least 10% of the total energy will be consumed by galaxies in need with existence expectancy index below 0,7.

Think of our galaxies as your “countries” (or how you call them??) and our population as citizens. We have similar healthcare and wellbeing characteristic as you, Hooomans”

“We are sending all the data to you right now. Let the data be with you, Hoomans… … …”

Transmission suddenly ends. You put your notebook and pencil away and start thinking. You really want to help this species optimize their well-being. You open up Python and upload the dataset from the narrowband radio signal receiver. It will be another great day at the observatory today.

————

* probably intergalactic species meant to say “humans” here but we will never know for sure

Description Data Recieved
The solutions are evaluated on two criteria: predicted future Index values and allocated energy from a newly discovered star

1) Index predictions are evaluated using RMSE metric

2) Energy allocation is also evaluated using RMSE metric and has a set of known factors that need to be taken into account.

Every galaxy has a certain limited potential for improvement in the index described by the following function:

Potential for increase in the Index = -np.log(Index+0.01)+3

Likely index increase dependent on potential for improvement and on extra energy availability is described by the following function:

Likely increase in the Index = extra energy * Potential for increase in the Index **2 / 1000

There are also several constraints:

in total there are 50000 zillion DSML available for allocation no galaxy should be allocated more than 100 zillion DSML or less than 0 zillion DSML galaxies with low existence expectancy index below 0.7 should be allocated at least 10% of the total energy available

3) Leaderboard is based on a combined scaled metric:

80% prediction task RMSE + 20% optimization task RMSE * lambda where lambda is a normalizing factor

4) Leaderboard is 80% public and 20% private

5) The submission should be in the following format:

Variable
Index
pred
out_pred
Description
Unique index from the test dataset in the ascending order
Prediction for the index on interest
Optimal energy allocation

#### COMPOSITE INDEX (TARGET) - a collection of great many equities, other securities, or other indexes that are averaged together to represent overall market or sector performance; it may be used in the natural or social sciences to summarize complex or multiddimentional data or rduntant measures.

###### Indexes: are statistical tools, which provide a useful measure of securities relative performance over time
###### Equity: value of shares issued...

### [MACHINE LEARNING]
#### Task 1: Use our data and train your model to predict this index with the highest possible level of certainty.
### [OPERATION RESEARCH]
#### Task 2: · To achieve the highest possible level of well-being through optimized allocation of additional energy

## TASK 1: MACHINE LEARNING PART

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import re
import tqdm
# import plotly.express as ex
import sklearn

#setting display options
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 500)
np.set_printoptions(linewidth =400)

from matplotlib import pyplot as plt
%matplotlib inline
#Advance-style plotting
import seaborn as sns
color =sns.color_palette()
sns.set_style('darkgrid')

#Ignore annoying warning from sklearn and seaborn
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
import datasist as ds
import datetime
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
import warnings
from itertools import cycle
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,auc,log_loss,roc_curve,roc_auc_score
import datetime
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import math
import re
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import gc
import warnings
warnings.filterwarnings('ignore')

In [24]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [27]:
print(train.shape,test.shape)

(334, 80) (890, 79)


In [28]:
train.head()

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),"Adolescent birth rate (births per 1,000 female creatures ages 15-19)",Total unemployment rate (female to male ratio),Vulnerable employment (% of total employment),"Unemployment, total (% of labour force)",Employment in agriculture (% of total employment),Labour force participation rate (% ages 15 and older),"Labour force participation rate (% ages 15 and older), female",Employment in services (% of total employment),"Labour force participation rate (% ages 15 and older), male",Employment to population ratio (% ages 15 and older),Jungle area (% of total land area),"Share of employment in nonagriculture, female (% of total employment in nonagriculture)",Youth unemployment rate (female to male ratio),"Unemployment, youth (% ages 15–24)","Mortality rate, female grown up (per 1,000 people)","Mortality rate, male grown up (per 1,000 people)","Infants lacking immunization, red hot disease (% of one-galactic year-olds)","Infants lacking immunization, Combination Vaccine (% of one-galactic year-olds)",Gross galactic product (GGP) per capita,"Gross galactic product (GGP), total","Outer Galaxies direct investment, net inflows (% of GGP)",Exports and imports (% of GGP),Share of seats in senate (% held by female),Natural resource depletion,"Mean years of education, female (galactic years)","Mean years of education, male (galactic years)","Expected years of education, female (galactic years)","Expected years of education, male (galactic years)","Maternal mortality ratio (deaths per 100,000 live births)",Renewable energy consumption (% of total final energy consumption),"Estimated gross galactic income per capita, male","Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),y
1806,1000000,UGCA 92,0.689316,65.638091,15088.072865,0.886753,15.136626,12.777039,0.751931,0.848223,157.189161,110.010529,95.886555,25.601782,714.297671,87.227101,121.475476,103.237307,11.307610,403.705408,58.626448,58

In [16]:
train.columns

Index(['galactic year', 'galaxy', 'existence expectancy index',
       'existence expectancy at birth', 'Gross income per capita',
       'Income Index', 'Expected years of education (galactic years)',
       'Mean years of education (galactic years)',
       'Intergalactic Development Index (IDI)', 'Education Index',
       'Intergalactic Development Index (IDI), Rank',
       'Population using at least basic drinking-water services (%)',
       'Population using at least basic sanitation services (%)',
       'Gross capital formation (% of GGP)', 'Population, total (millions)',
       'Population, urban (%)',
       'Mortality rate, under-five (per 1,000 live births)',
       'Mortality rate, infant (per 1,000 live births)',
       'Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64))',
       'Population, ages 15–64 (millions)',
       'Population, ages 65 and older (millions)',
       'Life expectancy at birth, male (galactic years)',
       'Life expect

#### galactic year: the duration of time required for the sun to orbit once around the center of the milky way galaxy

##### The galactic yeat provides a conveniently useable unit for depicting cosmic and geological time periods together;By contrast a 'billion year' scale does not allow for useful descrimination between geological events and a 'million year' scale requires some rather large numbers. Cool! since we have approximately and a million year; large numbers are needed.

In [17]:
train['galactic_years_count'] =train['galactic year'].map(train['galactic year'].value_counts())
test['galactic_years_count'] =train['galactic year'].map(test['galactic year'].value_counts())

In [18]:
#group galaxy by expectancy index, birth
#agg expectancy index alone

In [19]:
train.isna().sum()

galactic year                                                                                 0
galaxy                                                                                        0
existence expectancy index                                                                    1
existence expectancy at birth                                                                 1
Gross income per capita                                                                      28
Income Index                                                                                 28
Expected years of education (galactic years)                                                133
Mean years of education (galactic years)                                                    363
Intergalactic Development Index (IDI)                                                       391
Education Index                                                                             391
Intergalactic Development Index (IDI), R

### GDI

In [20]:
train['existence expectancy index'].fillna(method='bfill', inplace=True)
train['Income Index'].fillna(method='bfill', inplace=True)
train['Education Index'].fillna(method='bfill', inplace=True)
train['Gender Development Index (GDI)'].fillna(method='bfill', inplace=True)

In [21]:
train['ED/IDindex'] = 0.3333333*(train['Education Index']) + 0.3333333*(train['Income Index'])
train['ED/IDindexLCM'] =train['ED/IDindex'] * 3
train['life expectancy index'] =(train['Gender Development Index (GDI)'] *3) - (train['ED/IDindexLCM'])

In [22]:
train.isna().sum()

galactic year                                                                                 0
galaxy                                                                                        0
existence expectancy index                                                                    0
existence expectancy at birth                                                                 1
Gross income per capita                                                                      28
Income Index                                                                                  0
Expected years of education (galactic years)                                                133
Mean years of education (galactic years)                                                    363
Intergalactic Development Index (IDI)                                                       391
Education Index                                                                               0
Intergalactic Development Index (IDI), R

In [ ]:
%history

In [ ]:
#### Galaxy: is a gravitationally bound system of stars, stellar remnants, interstellar gas, dust and dark matter

In [ ]:
train['galaxy_appearance'] =train.galaxy.map(train.galaxy.value_counts())
test['galaxy_appearance'] =test.galaxy.map(test.galaxy.value_counts())

In [ ]:
train[['galactic year','galaxy_years_apperance' ]]

In [ ]:
def threshold(x):
    if x >= 0.48:
        return 1
    else:
        return 0
train['y_threshold']= train['y'].apply(threshold)

In [ ]:
train['y_threshold'].value_counts().to_frame()

In [ ]:
Yes = len(train[train.y_threshold ==1])
No = len(train[train.y_threshold ==0])
Total = len(train)
print ('There are imbalanace datset with a %i/%i ratio'%((No/Total*100), (Yes/Total*100)+1))

the train_test_split method considered purely random sampling method
this is generally fine if your dataset is large enough (hundreds of thousands or even millions rows) but if it’s not, you run the risk of introducing a significant sampling bias and you do not really want that
And the goal of this project is not to build models but to create a bias free testset. now let’s get down to work.


In [ ]:
#NB: Galaxies(as country)

“Only two steps prevent us from achieving singularity

· To understand what makes us better off.

Our elders used the composite index to measure our well-being performance, but this knowledge has disappeared in the sands of time.

Use our data and train your model to predict this index with the highest possible level of certainty.

· To achieve the highest possible level of well-being through optimized allocation of additional energy

We have discovered the star of an unusually high energy of 50000 zillion DSML.

We have agreed between ourselves that 

· no one galaxy will consume more than 100 zillion DSML 

and 

· at least 10% of the total energy will be consumed by galaxies in need with existence expectancy index below 0,7

In [ ]:
corr = train.corr();corr['y'].sort_values(ascending=False)

In [ ]:
train.galaxy.value_counts().to_frame()

In [ ]:
test=test.drop(['y'], axis=1)

In [ ]:
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)

In [ ]:
# Create our model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=300)

# Split the data into inputs (X) and desired output (y)
X, y = train[test.columns[2:]], train['y']

# Fit the model
model.fit(X, y)

In [ ]:
# Score the model
from sklearn.metrics import mean_squared_error
mean_squared_error(y, model.predict(X), squared=False)

In [ ]:
preds = model.predict(test[test.columns[2:]])
test['y'] = preds
test.head()

In [ ]:
ss = pd.DataFrame({
    'Index':test.index,
    'pred':test['y'],
    'opt_pred':0
})
ss.head()


It has our predictions, but we still need to deal with opt_pred. The competition states that:

Countries with low existence expectancy index need at least 10% of the 50,000 available energy thingees.
No country gets more than 100 energy thingees

We'll try to optimise this all late, but for now le's do the bare minimum to submit:
For the low-EEI places we'll allocate 99 energy things each (more than 10% of the total, and less than 100). For the other places we'll give each a fraction of the remainder:


A better attempt at the optimizing part
We're told:
Potential for increase in the Index = -np.log(Index+0.01)+3
Likely increase in the Index = extra energy * Potential for increase in the Index **2 / 1000
In other words, allocating energy to the ones with the highest potential for increase will net a larger improvement overall. But we need to give some to the poor galaxies with low EEI. 
With our predicted index (y) we can calculate the potential for increase as a new column and plot it:

In [ ]:
ss = pd.DataFrame({
    'Index':test.index,
    'pred':test['y'],
    'opt_pred':0, 
    'eei':test['existence expectancy index'] # So we can split into low and high EEI galaxies
})
# Fix opt_pred
n_low_eei = ss.loc[ss.eei < 0.7].shape[0]
n_high_eei = ss.loc[ss.eei >= 0.7].shape[0]
ss.loc[ss.eei < 0.7, 'opt_pred'] = 99 # 66*99 = 6534 - >10%, <100 each
ss.loc[ss.eei >= 0.7, 'opt_pred'] = (50000-6534)/n_high_eei # The rest to high eei gs
# Leaving 5k zillion whatsits to the admin
ss = ss.drop('eei', axis=1)
ss.to_csv('Sub_test.csv', index=False)
ss.head()

In [ ]:
ss = pd.DataFrame({
    'Index':test.index,
    'pred':test['y'],
    'opt_pred':0, 
    'eei':test['existence expectancy index'] # So we can split into low and high EEI galaxies
})
# Fix opt_pred
n_low_eei = ss.loc[ss.eei < 0.7].shape[0]
n_high_eei = ss.loc[ss.eei >= 0.7].shape[0]
ss.loc[ss.eei < 0.7, 'opt_pred'] = 92 # 66*99 = 6534 - >10%, <100 each
ss.loc[ss.eei >= 0.7, 'opt_pred'] = (50000-6534)/n_high_eei # The rest to high eei gs
# Leaving 5k zillion whatsits to the admin
ss = ss.drop('eei', axis=1)
ss.to_csv('Sub_test.csv', index=False)
ss.head()

In [ ]:
ss.opt_pred.max()

In [ ]:
test['potential_increase'] = -np.log(test['y']+0.01)+3
test['potential_increase'].plot(kind='hist')

In [ ]:
test.plot(x='existence expectancy index', y='potential_increase', kind='scatter') # Low EEI have higer potential for increase!

In [ ]:
train.columns

In [ ]:
ds.structdata.check_train_test_set(train, test,'galactic year', 'Income Index')

In [ ]:
train.galaxy.value_counts()

In [ ]:
sns.scatterplot(x='Income Index', y='Income Index', data =train)

In [ ]:
train['Income Index'].min()

In [ ]:
ds.structdata.display_missing(train)

# UNIVARIATE ANALYSIS

In [ ]:
train['galactic year'].unique()

In [ ]:
test.shape